In [1]:
import os
import sys
sys.path.append('../../')
from recs.backtest import get_backtest_answer
from recs.utils.path import *
import __init__
from recs.utils.common import predict_ranking, readjson2dict, recall_evaluate, id2cat
import recs
from tqdm import tqdm
import pandas as pd
tqdm.pandas()

In [2]:
u2idx = readjson2dict("crm_idx")
tran_data = pd.read_pickle(DATA_PATH / "trans_buy.pkl")
real = get_backtest_answer(tran_data, start=20200000, end=20210000)

100%|██████████| 28496/28496 [01:15<00:00, 378.26it/s]


In [3]:
target_date = 202000
trans_buy = tran_data[tran_data.buy_date < (target_date)*100]
crm = pd.read_csv(DATA_PATH / "crm_for_clustering_2019.csv")
crm = crm[crm.yyyymm == 201912]
crm['身分證字號'] = crm['身分證字號'].apply(lambda x: int(u2idx.get(x, -1)))
crm = crm[crm['身分證字號'] > -1].drop(columns='index')

# print(crm.columns)
target_users = crm[crm.yyyymm == 201912]['身分證字號'].unique().tolist()
best_sells = trans_buy[
    (trans_buy.buy_date > 20171200 )
].fund_id.value_counts().rename_axis('fund_id').reset_index(name='count')
trans_buy

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (29) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id_number,fund_id,buy_date,yyyymm
1,13627,930,20170327,201702
5,13651,748,20190709,201906
8,13651,2053,20170926,201708
11,13653,2518,20170616,201705
12,13654,2091,20160322,201602
...,...,...,...,...
395706,67934,3041,20180116,201712
395707,67951,2108,20191114,201910
395708,67969,482,20180509,201804
395709,67971,2058,20180515,201804


In [ ]:
### for VAE
# vaecf = recs.models.VAECF(
#     k=10,
#     autoencoder_structure=[20],
#     act_fn="tanh",
#     likelihood="mult",
#     n_epochs=100,
#     batch_size=100,
#     learning_rate=0.001,
#     beta=1.0,
#     seed=123,
#     use_gpu=True,
#     verbose=True,
# )
# trans_buy['t_weight'] = trans_buy.buy_date.progress_apply(lambda x: pow((x - trans_buy.buy_date.min() + 1) / (trans_buy.buy_date.max() - trans_buy.buy_date.min()), 2))
# rate_df = trans_buy.groupby(["id_number", "fund_id"]).agg({'t_weight':'sum'}).reset_index().rename(columns={"t_weight": "rating"})
# uir_data = rate_df[['id_number', 'fund_id', 'rating']].values
# train_set = recs.datasets.Dataset.from_uir(uir_data, seed=41)
# df = pd.DataFrame(columns=['userId', 'itemId', 'rating'], data=uir_data)

# vaecf.fit(train_set)
# pred = predict_ranking(vaecf, df, 'userId', 'itemId', 'rating', False)
# pred.sort_values(by='rating', ascending=False, inplace=True)
# pred



In [ ]:
### for VAE
# rec_dict = {}
# for u in tqdm(target_users):
#     temp = pred[pred.userId == u]
#     if len(temp) == 0:
# #         ignore_items = trans_buy[trans_buy.id_number == u].fund_id.unique().tolist()        
# #         rec_dict[u] = best_sells[~best_sells['fund_id'].isin(ignore_items)]['fund_id'].tolist()[:10]
#         rec_dict[u] = best_sells['fund_id'].tolist()[:10]
#     else:
#         rec_dict[u] = temp.itemId.unique().tolist()[:10]
# for i in tqdm(rec_dict):
#     rec_dict[i] = [int(x) for x in rec_dict[i]]

In [ ]:
# import json
# from recs.utils.common import convert_result_json_2_csv
# r_name = "rb_2020_vaecf"
# json_name = r_name+".json"
# json_save_path = RESULT_PATH / json_name 
# # with open(json_save_path, 'w') as jf:
# #     json.dump(rec_dict, jf)

# csv_name = f"{r_name}.csv"
# save_path = RESULT_PATH / csv_name
# # convert_result_json_2_csv(str(json_save_path), str(save_path))

# vaecf_data = pd.read_csv(save_path)

In [4]:
# valid data
tran_data = pd.read_pickle(DATA_PATH / "trans_buy.pkl")
user2020 = list(tran_data[tran_data.buy_date > 20200000].id_number.unique())
user2017 =list(tran_data[tran_data.buy_date < 20200000].id_number.unique())
new_user = list(set(user2020) - set(user2017))
new_user = list(set(target_users) & set(new_user))
new_real = {}
for i in tqdm(real):
    if i in new_user:
        new_real[i] = real[i]

print(len(new_real))

100%|██████████| 28496/28496 [00:09<00:00, 2919.59it/s]

2112


In [6]:
best_sell_rec = {}
for i in new_user:
    best_sell_rec[i] =  best_sells['fund_id'][:10]
recall_evaluate(best_sell_rec, new_real)

100%|██████████| 2112/2112 [00:00<00:00, 9748.26it/s]


{'recall@3': 0.04273852594165095,
 'recall@5': 0.05649999595312097,
 'recall@10': 0.06894099030552821,
 'recall@all': 0.06894099030552821}

In [49]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from sklearn.cluster import Birch, DBSCAN
import itertools
import numpy as np
import umap


n_cluster = 50
n_comp = 10
mx = 0
mx_ncl = 0
mx_nco = 0

def findbest(n_cluster, n_comp, feature_select="umap", clustering="birch"):
    user_data = crm[crm.yyyymm == 201912]
    udata = user_data.drop(columns=['身分證字號', 'CIFAOCODE'])
    udata = udata.replace(-np.inf, -1)
    udata.fillna(0, inplace=True)

    if feature_select not in ["tsne", "umap"]:
        raise ValueError("feature select method not allowed")
    
    if feature_select == "tsne":
        x_embedded = TSNE(n_components=3, init='random', random_state=47, n_jobs=30).fit_transform(udata.values)
        x_embedded = np.array(x_embedded)
    
    if feature_select == "umap":
        x_embedded = umap.UMAP(n_components=n_comp).fit_transform(udata.values)

    if clustering not in ["birch", "dbscan"]:
        raise ValueError("clustering method not allowed")

    if clustering == "birch":
        brc = Birch(n_clusters=n_cluster)
        brc.fit(x_embedded)
        brc_x = np.array(brc.predict(x_embedded))
        user_data['cluster'] = brc_x
        
    if clustering == "dbscan":
        

    tran_data = trans_buy.copy()


    cluster_rec = {}
    for i in tqdm(range(n_cluster)):
        userlen = len(user_data[user_data['cluster'] == i])
        tranlen = len(tran_data[tran_data['id_number'].isin( user_data[user_data['cluster'] == i]['身分證字號'])])
        temp = tran_data[tran_data['id_number'].isin( user_data[user_data['cluster'] == i]['身分證字號'])]['fund_id'].value_counts().index.tolist()[:10]
        cluster_rec[i] = temp

    cluster_rec_dict = {}
    for u in tqdm(target_users):
        temp = user_data[user_data['身分證字號']==u]
        t_cluster = temp.cluster.tolist()[0]    
        cluster_rec_dict[u] = cluster_rec[t_cluster]

    temp_dict = {}
    for i in cluster_rec_dict:
        temp_dict[int(i)] = best_sells['fund_id'][:10]

    
    r = recall_evaluate(cluster_rec_dict, new_real)
    return r['recall@10'], n_cluster, n_comp
    
        

for i in range(10):
    for j in range(20):
        r10, n_clu, n_com = findbest(5*(i+11), 3*(j+1))
        if r10 > mx:
            mx = r10
            mx_ncl = n_clu
            mx_nco = n_com
            print(mx, mx_ncl, mx_nco)
    

/usr/local/lib/python3.6/dist-packages/sklearn/manifold/_spectral_embedding.py:236: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"
100%|██████████| 11056/11056 [00:00<00:00, 269516.64it/s]


0.018578483271981444 55 3


/usr/local/lib/python3.6/dist-packages/sklearn/manifold/_spectral_embedding.py:236: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"
100%|██████████| 11056/11056 [00:00<00:00, 265451.34it/s]
/usr/local/lib/python3.6/dist-packages/sklearn/manifold/_spectral_embedding.py:236: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"
100%|██████████| 11056/11056 [00:00<00:00, 274468.20it/s]
/usr/local/lib/python3.6/dist-packages/sklearn/manifold/_spectral_embedding.py:236: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"
100%|██████████| 11056/11056 [00:00<00:00, 274135.57it/s]


0.018618898014820213 55 12


/usr/local/lib/python3.6/dist-packages/sklearn/manifold/_spectral_embedding.py:236: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"
100%|██████████| 11056/11056 [00:00<00:00, 268810.47it/s]
/usr/local/lib/python3.6/dist-packages/sklearn/manifold/_spectral_embedding.py:236: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"
100%|██████████| 11056/11056 [00:00<00:00, 268343.81it/s]
/usr/local/lib/python3.6/dist-packages/sklearn/manifold/_spectral_embedding.py:236: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"
100%|██████████| 11056/11056 [00:00<00:00, 273795.67it/s]
/usr/local/lib/python3.6/dist-packages/sklearn/manifold/_spectral_embedding.py:236: UserWarning: Graph is not

...

/usr/local/lib/python3.6/dist-packages/sklearn/manifold/_spectral_embedding.py:236: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"
100%|██████████| 11056/11056 [00:00<00:00, 277294.43it/s]


0.019602921629089845 85 57


/usr/local/lib/python3.6/dist-packages/sklearn/manifold/_spectral_embedding.py:236: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"
100%|██████████| 11056/11056 [00:00<00:00, 226558.52it/s]
/usr/local/lib/python3.6/dist-packages/sklearn/manifold/_spectral_embedding.py:236: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"
100%|██████████| 11056/11056 [00:00<00:00, 274231.22it/s]
/usr/local/lib/python3.6/dist-packages/sklearn/manifold/_spectral_embedding.py:236: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"
100%|██████████| 11056/11056 [00:00<00:00, 275013.49it/s]
/usr/local/lib/python3.6/dist-packages/sklearn/manifold/_spectral_embedding.py:236: UserWarning: Graph is not

100%|██████████| 11056/11056 [00:00<00:00, 275446.38it/s]
/usr/local/lib/python3.6/dist-packages/sklearn/manifold/_spectral_embedding.py:236: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"
100%|██████████| 11056/11056 [00:00<00:00, 274218.25it/s]
/usr/local/lib/python3.6/dist-packages/sklearn/manifold/_spectral_embedding.py:236: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"
100%|██████████| 11056/11056 [00:00<00:00, 274464.95it/s]
/usr/local/lib/python3.6/dist-packages/sklearn/manifold/_spectral_embedding.py:236: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"
100%|██████████| 11056/11056 [00:00<00:00, 267218.09it/s]
/usr/local/lib/python3.6/dist-packages/sklearn/mani

100%|██████████| 11056/11056 [00:00<00:00, 276250.42it/s]
/usr/local/lib/python3.6/dist-packages/sklearn/manifold/_spectral_embedding.py:236: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"
100%|██████████| 11056/11056 [00:00<00:00, 258017.67it/s]
/usr/local/lib/python3.6/dist-packages/sklearn/manifold/_spectral_embedding.py:236: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"
100%|██████████| 11056/11056 [00:00<00:00, 269308.47it/s]
/usr/local/lib/python3.6/dist-packages/sklearn/manifold/_spectral_embedding.py:236: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"
100%|██████████| 11056/11056 [00:00<00:00, 270500.81it/s]
/usr/local/lib/python3.6/dist-packages/sklearn/mani

In [50]:
# umap birch 0.01971914093398498 50 12
# umap birch 0.019602921629089845 85 57
print(mx, mx_ncl, mx_nco)

0.019602921629089845 85 57


In [ ]:
findbest(50, )

In [31]:
# umap 5

temp_dict = {}
for i in cluster_rec_dict:
    temp_dict[int(i)] = best_sells['fund_id'][:10]
    
print(recall_evaluate(cluster_rec_dict, new_real))
print(recall_evaluate(cvae_rec_dict, new_real))
print(recall_evaluate(temp_dict, new_real))

100%|██████████| 11056/11056 [00:00<00:00, 263764.80it/s]


{'recall@3': 0.009016885841686856, 'recall@5': 0.011629382787577434, 'recall@10': 0.018530100435971978, 'recall@all': 0.018530100435971978}


100%|██████████| 11056/11056 [00:00<00:00, 314750.73it/s]


{'recall@3': 0.0, 'recall@5': 0.0, 'recall@10': 0.0, 'recall@all': 0.0}


100%|██████████| 11056/11056 [00:00<00:00, 42398.39it/s]

{'recall@3': 0.00816423360969309, 'recall@5': 0.010793052772520937, 'recall@10': 0.013169624776164578, 'recall@all': 0.013169624776164578}
